# Demo: Creating CAS Tables with the copyTable Action
**Note:** The [table.copyTable](https://go.documentation.sas.com/doc/en/pgmsascdc/v_006/caspg/cas-table-copytable.htm?homeOnFail) action was introduced in SAS Viya 2020.1. For previous releases, the [table.partition](https://go.documentation.sas.com/doc/en/pgmsascdc/9.4_3.5/caspg/cas-table-partition.htm) action is required.

### 1. Import Packages and Connect to the CAS Server

Visit the documentation for the SWAT [(SAS Scripting Wrapper for Analytics Transfer)](https://sassoftware.github.io/python-swat/index.html) package.

In [ ]:
## Import packages
import swat
import pandas as pd
import matplotlib.pyplot as plt

## Set options
pd.set_option('display.max_columns', None)

## Connect to CAS
conn = swat.CAS('server.demo.sas.com', 30571, 'student', 'Metadata0', name = 'py03d06')

## Function to load the loans_raw.sashdat file into memory if necessary
def loadLoans():
    conn.loadTable(path = 'loans_raw.sashdat', caslib = 'PIVY',
                   casOut = {'name' : 'loans_raw',
                            'caslib' : 'casuser',
                            'promote' : True})

### 2. Explore Available CAS Tables and Data Source Files


a. Use the tableInfo action to view all available in-memory tables in the **Casuser** caslib. If the **LOANS_RAW** CAS table is not available, uncomment the loadLoans function and execute the cell.

In [ ]:
#loadLoans()
conn.tableInfo(caslib = 'casuser')

b. Reference the **LOANS_RAW** CAS table.

In [ ]:
tbl = conn.CASTable('loans_raw', caslib = 'casuser')

### 3. Quick Exploration

a. Preview the **LOANS_RAW** table using the head method.

In [ ]:
tbl.head()

b. Explore the **CancelledReason** column. Notice that truncated values exists. Take note to fix this during the data preparation stage.

In [ ]:
(tbl
 .CancelledReason
 .value_counts())

### 4. Create a New CAS Table

Create a new CAS table from the **LOANS_RAW** table that completes the following:
- Select rows where **Category** is *Credit Card*
- Create two new calculated columns:
    - **CancelledReason_fix** - Fixes the truncation issues in the **CancelledReason** column.
    - **OpenCCDate** - Create a credit card open date column by using the **Year**, **Month** and **Day** columns.
- Keep the **AccNumber**, **Month**, **Day**, **Year**, **CCOpenDate**, **Amount**, **InterestRate**, **LoanGrade**, **LoanStatus**, **LastPurchase**, **Cancelled**, and **CancelledReason_fix** columns.
- Name the new CAS table **CreditCard** and place it in the **Casuser** caslib.

a. Add the where parameter to subset the **LOANS_RAW** CAS table for only rows where **Category** equals *Credit Card*. View the new **CASTable** object to confirm that the where parameter was added.

In [ ]:
tbl.where = 'Category = "Credit Card"'
tbl

b. Create two new computed columns using the computedVarsProgram parameter. Instead of adding the entire string in the computedVarsProgram parameter, create two new string variables with the calculated column expressions. 

**Note**: The LENGTH statement is used to set a specific length of the CAS table column. If the LENGTH statement is omitted, SAS determines the length based on the first assignment statement.

In [ ]:
## Use a SAS IF-THEN/ELSE statement to fix the CancelledReason column. If you do not specify the length...
fixCancelled = '''length CancelledReason_fix varchar(18);
                   if CancelledReason = "Better Altern" then CancelledReason_fix = "Better Alternative";
                     else if CancelledReason = "No Longer Nee" then CancelledReason_fix = "No Longer Needed";
                     else CancelledReason_fix = CancelledReason;
               '''

## Use an assignment statement and the SAS MDY function to create a SAS date
createCCOpenDate = "CCOpenDate = mdy(Month,Day,Year);"

## Add the computed columns expressions to the computedVarsProgram parameter
tbl.computedVarsProgram = fixCancelled + createCCOpenDate
tbl

c. Use the head method to preview the table and confirm that the CAS table is calculating the new columns. Use the value_counts method to confirm that the **CancelledReason_fix** column is correct.

In [ ]:
df = tbl.head()

cancelled_df = (tbl
                .CancelledReason_fix
                .value_counts())

display(df, cancelled_df)

d. Add the required columns for the new table using the vars parameter. Then execute the head method on the **CASTable** object to preview the new table. Notice that the **CCOpenDate** column contains a numeric SAS date value. A format must be applied on the column to make the date human readable. That will be added later.

In [ ]:
## Keep the following columns
tbl.vars = ['AccNumber', 'Month', 'Day', 'Year', 'CCOpenDate', 'Amount', 'InterestRate', 
            'LoanGrade', 'LoanStatus', 'LastPurchase', 'Cancelled', 'CancelledReason_fix']

## Preview the table
tbl.head(10)

e. Specify a dictionary with the output information of the new CAS table in the variable **ccOutTbl**. Here, the new table is named **CreditCard**, it is placed in the **Casuser** caslib, it is replaced if it already exists, and a label is added to the table.

In [ ]:
ccOutTbl = dict(name = 'CreditCard', 
                caslib = 'casuser', 
                replace = True, 
                label = 'All credit card loan data cleaned')

f. Execute the copyTable action on the input CAS table **tbl**, and then specify the casOut parameter with the dictionary **ccOutTbl** with the information for the output CAS table. The copyTable action returns a **CASResults** object with information about the newly created table.

In [ ]:
tbl.copyTable(casOut = ccOutTbl)

g. Use the tableInfo action to check the **Casuser** caslib for the new table. Notice that the new table **CreditCard** has been created.

In [ ]:
conn.tableInfo(caslib = 'casuser')

h. Reference and preview the new CAS table **creditcard**. 

In [ ]:
ccTbl = conn.CASTable('creditcard', caslib = 'casuser')
ccTbl.head()

### 5. Adding a SAS Date Format

a. To add a SAS date format to the data value that we saw above, you can apply SAS format. There are multiple ways to add a format to a column. One way is to use the [table.alterTable](https://documentation.sas.com/doc/en/pgmsascdc/v_017/caspg/cas-table-altertable.htm?homeOnFail) action to apply a format to a column. Here the alterTable action is used to apply the SAS *mmddyy* format to the **CCOpenDate** column.

In [ ]:
conn.alterTable(name = 'creditcard', caslib = 'casuser',
                columns = [
                    {'name' : 'CCOpenDate', 'format' : 'mmddyy10.'}
                ])

## Preview the table with the format
ccTbl.head()

b. Use the columnInfo action to view the CAS table column information. Notice that the **CCOpenDate** column now contains a SAS date format.

In [ ]:
ccTbl.columnInfo()

### 6. Terminate the CAS Session

It's best practice to always terminate the CAS session when you are done.

In [ ]:
conn.terminate()